In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import seaborn as sns

In [3]:
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
df.dropna(inplace=True)
df.columns = ['status', 'stay-time', 'Aspect-ratio', 'Density', 'Dis Variance', 'Velocity Mean', 'Velocity Variance',
              'a-Variance', 'maxDis', 'head-Variance', 'Class label']
df.head(5) 

,status,stay-time,Aspect-ratio,Density,Dis Variance,Velocity Mean,Velocity Variance,a-Variance,maxDis,head-Variance,Class label
0,1.0,5.520556,8.594854,0.024205,3.278965,0.014234,0.008460,0.000056,167.689320,32.164128,1
1,1.0,3.850833,3.030676,0.051397,5.937132,0.034289,0.027529,0.000134,66.725869,5.636387,1
2,1.0,3.850556,6.788039,0.041447,3.433945,0.023077,0.014594,0.000103,110.791868,7.544310,1
3,1.0,4.348889,6.200071,0.011759,7.810434,0.029176,0.024673,0.000097,192.090427,9.990709,1
4,1.0,2.199444,2.410629,0.044884,5.714460,0.033267,0.025093,0.000141,45.923765,4.390358,1


In [5]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
X = df.iloc[:,0:10]
y = df.iloc[:,10]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = SVC(C=2, gamma=0.001, kernel='rbf').fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy)


print('***********')
scores = cross_val_score(model, X_train, y_train, cv=10)
print(scores)  # 打印输出每次迭代的度量值（准确度）
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))  # 获取置信区间。（也就是均值和方差）
print()
scores = cross_val_score(model, X_train, y_train, cv=10)  
print('准确率：',np.mean(scores), scores)  
precisions = cross_val_score(model, X_train, y_train, cv=10, scoring='precision')  
print('精确率：', np.mean(precisions), precisions)  
recalls = cross_val_score(model, X_train, y_train, cv=10, scoring='recall')  
print('召回率：', np.mean(recalls), recalls)  
f1s = cross_val_score(model, X_train, y_train, cv=10, scoring='f1')  
print('综合评价指标：', np.mean(f1s), f1s)  

#Best: -0.213943 using {'C': 2, 'gamma': 0.001, 'kernel': 'rbf'}

0.9310170835435738
***********
[0.93514515 0.93020383 0.9382144  0.93327155 0.9304912  0.93666976
 0.92678406 0.93327155 0.93108776 0.93170581]
Accuracy: 0.933 (+/- 0.006)

准确率： 0.9326845055480835 [0.93514515 0.93020383 0.9382144  0.93327155 0.9304912  0.93666976
 0.92678406 0.93327155 0.93108776 0.93170581]
精确率： 0.9515948131325429 [0.95251555 0.9536348  0.955342   0.95184136 0.95159453 0.95521542
 0.94213483 0.9533826  0.95368782 0.94659921]
召回率： 0.9268278262673473 [0.93042518 0.91993374 0.93318609 0.92766427 0.92269464 0.93042518
 0.92600773 0.92600773 0.92154696 0.93038674]
综合评价指标： 0.939039484882646 [0.94134078 0.93648117 0.94413408 0.93959732 0.93692178 0.94265734
 0.93400167 0.9394958  0.93734195 0.93842296]


In [ ]:
# Tune learning_rate
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
# load data
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
# split data into X and y
X = df.iloc[:,0:10]
Y = df.iloc[:,10]
# grid search
model = SVC(probability=True)
#param_grid = dict(learning_rate=learning_rate, n_estimators=[10, 50, 100], max_depth=np.arange(3, 8), min_child_weight=np.arange(3, 8), subsample=np.arange(5, 9)/10)
#param_grid = dict(kernel=['rbf'], gamma=[1e-3], C=np.arange(1,10))
param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 2, 3, 4, 5]},
                    {'kernel': ['linear'], 'C': [1, 2, 3, 4, 5]}]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) #n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with: %r" % (mean, param))
